In [1]:
from distutils.dir_util import copy_tree

from_dir = '/kaggle/input/taco-with-hard-negatives'
to_dir = '/kaggle/working/'

copy_tree(from_dir, to_dir)


['/kaggle/working/TACO_with_hard_negatives/data.yaml',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/1000061_jpg.rf.7779d6ce157b1c650648a2e9de0b5d38_webcam_1.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/3IMG_4876_JPG.rf.e01212e9b631ca54cda9a0fc935ddc9c_webcam_2.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/9000020_jpg.rf.a311c90f84f41a91a12535ba484663de_webcam_4.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/1000127_JPG.rf.da5be93c943766123144a46a38675088_webcam_0.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/Places365_val_00000498.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/reusable-cup-with-paper-straw-womans-handzero-waste-lifestyle_377884-382_jpg.rf.e2a66ede5b35617e5f74c710f69617a5_webcam_4.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/Places365_val_00000189.txt',
 '/kaggle/working/TACO_with_hard_negatives/valid/labels/4000034_JPG.rf.cb1f16a29cffe1e2258db8dc0a62355f.txt',
 '/

In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3

In [3]:
#converting to object detection dataset
import os

base_dir = "/kaggle/working/TACO_with_hard_negatives"  

subsets = ['train', 'test', 'valid']

def polygon_to_bbox(points):
    """Convert list of (x, y) points to bounding box in YOLO format"""
    xs = points[::2]
    ys = points[1::2]
    x_min = min(xs)
    x_max = max(xs)
    y_min = min(ys)
    y_max = max(ys)

    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    width = x_max - x_min
    height = y_max - y_min

    return x_center, y_center, width, height

for subset in subsets:
    label_dir = os.path.join(base_dir, subset, 'labels')
    for fname in os.listdir(label_dir):
        if not fname.endswith(".txt"):
            continue

        file_path = os.path.join(label_dir, fname)
        new_lines = []

        with open(file_path, 'r') as f:
            for line in f:
                parts = list(map(float, line.strip().split()))
                class_id = int(parts[0])
                polygon_points = parts[1:]
                x_center, y_center, width, height = polygon_to_bbox(polygon_points)
                new_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                new_lines.append(new_line)

        # Overwrite the file with detection-format annotations
        with open(file_path, 'w') as f:
            f.write('\n'.join(new_lines) + '\n')


In [4]:
from ultralytics import RTDETR

model = RTDETR('/kaggle/input/rtdetr/pytorch/default/1/rtdetr_last.pt')  

model.train(
    data='/kaggle/working/TACO_with_hard_negatives/data.yaml',
    name='v2_rtdetr_continued',
    resume=True,
    epochs=300,
    batch=-1,
    device=0,
    imgsz=640,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.9,
    weight_decay=0.01,
    warmup_epochs=3.0,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    cos_lr=True,
    amp=True,
    val=True,
    patience=50,
    seed=42,
    deterministic=True,
    save=True,
    plots=True,
    overlap_mask=True,
    mask_ratio=4,
    freeze=10,
    dropout=0.2,

    #  Spatial augmentations
    degrees=20.0,
    translate=0.1,
    scale=0.8,
    shear=10.0,
    perspective=0.0005,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    cutmix=0.2,
    copy_paste=0.3,


)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.159 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=True, cutmix=0.2, data=/kaggle/working/TACO_with_hard_negatives/data.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

100%|██████████| 755k/755k [00:00<00:00, 17.4MB/s]


WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5, 6, True, True]
  7                  -1

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.8MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2304.8±1132.4 MB/s, size: 261.9 KB)


...

  0%|          | 0/401 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:92.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
    298/300      7.93G     0.5163      0.458     0.1899          0        640: 100%|██████████| 401/401 [03:45<00:00,  1.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:10<00:00,  2.43it/s]


                   all        456       1325      0.815      0.598      0.639      0.386
EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 248, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

113 epochs completed in 7.369 hours.
Optimizer stripped from runs/detect/v2_rtdetr/weights/last.pt, 66.2MB
Optimizer stripped from runs/detect/v2_rtdetr/weights/best.pt, 66.2MB

Validating runs/detect/v2_rtdetr/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 302 layers, 32,035,115 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:12<00:00,  2.12it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        456       1325      0.798       0.61       0.64      0.389
             Cigarette         37         97      0.593      0.346      0.374      0.164
        Plastic bottle         53         93      0.848      0.677      0.716      0.464
       Plastic wrapper         88        142      0.838      0.582       0.62      0.332
     Plastic container         40         46      0.799      0.739      0.794      0.513
           Plastic cup         22         25       0.73       0.52      0.535      0.396
       Plastic lid/cap         55         67      0.863      0.755      0.776      0.497
       Plastic utensil         20         22       0.79      0.864       0.87      0.506
         Plastic straw         19         32      0.678      0.312      0.366      0.192
           Plastic bag         43         54      0.878      0.833      0.856      0.553
        Plastic gloves          8          8      0.991          1      0.995      0.641
        Foam containe

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7cfc9ebe9f50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,  